## Youtube에서 썸네일 URL과 title 긁어와서 csv파일로 저장

In [2]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser
import pandas as pd

# https://console.cloud.google.com/apis/credentials 여기서 API발급받아 사용
DEVELOPER_KEY='' # 내 API 키값 입력
YOUTUBE_API_SERVICE_NAME='youtube'
YOUTUBE_API_VERSION='v3'

youtube=build(YOUTUBE_API_SERVICE_NAME,YOUTUBE_API_VERSION,developerKey=DEVELOPER_KEY)

In [7]:
Channels=["Zane Travel", "Indigo Traveller","ON World Travel"]
video_names=[]
video_ids=[]
url=[]
for Channel in Channels:
    search_response=youtube.search().list(
        q=Channel,
        order='relevance',
        part='snippet',
        maxResults=50,
    ).execute()
#     print(search_response['items'])
    channel_id=search_response['items'][0]['id']['channelId']
    channel_playlists=youtube.playlists().list(
        channelId=channel_id,
        part='snippet',
        maxResults=50).execute()
    print(f"Playlist num of {Channel}: {len(channel_playlists['items'])}")
    ids=[]
    titles=[]
    for i in channel_playlists['items']:
        ids.append(i['id'])
        titles.append(i['snippet']['title'])
    df=pd.DataFrame([ids,titles]).T
    df.columns=['PlayLists','Titles']
    for n in range(0,len(channel_playlists['items'])):
#         print(f"{Channel}'s PlayList{n}->{len(playlistitems_list_response['items'])} videos processing ...")
        dtcu=df['PlayLists'][n] # 0,1,2
        playlist_videos=youtube.playlistItems().list(
            playlistId=dtcu,
            part='snippet',
            maxResults=50)
        playlistitems_list_response=playlist_videos.execute()
        print(f"{Channel}'s PlayList{n}->{len(playlistitems_list_response['items'])} videos processing ...")
#         print(len(playlistitems_list_response['items']))
        for v in playlistitems_list_response['items']:
            if(v['snippet']['thumbnails']=={}):
                continue
            else:
                try:
                    url.append(v['snippet']['thumbnails']['standard']['url'])
                except:
                    url.append(v['snippet']['thumbnails']['high']['url'])
            video_names.append(v['snippet']['title'])

    
vdf=pd.DataFrame([url,video_names]).T
vdf.columns=['Thumbnail URL','Title']
    

Playlist num of Zane Travel: 3
Zane Travel's PlayList0->8 videos processing ...
Zane Travel's PlayList1->26 videos processing ...
Zane Travel's PlayList2->24 videos processing ...
Playlist num of Indigo Traveller: 44
Indigo Traveller's PlayList0->4 videos processing ...
Indigo Traveller's PlayList1->4 videos processing ...
Indigo Traveller's PlayList2->6 videos processing ...
Indigo Traveller's PlayList3->6 videos processing ...
Indigo Traveller's PlayList4->6 videos processing ...
Indigo Traveller's PlayList5->5 videos processing ...
Indigo Traveller's PlayList6->5 videos processing ...
Indigo Traveller's PlayList7->3 videos processing ...
Indigo Traveller's PlayList8->5 videos processing ...
Indigo Traveller's PlayList9->8 videos processing ...
Indigo Traveller's PlayList10->9 videos processing ...
Indigo Traveller's PlayList11->11 videos processing ...
Indigo Traveller's PlayList12->8 videos processing ...
Indigo Traveller's PlayList13->6 videos processing ...
Indigo Traveller's Pla

In [8]:
vdf

,Thumbnail URL,Title
0,https://i.ytimg.com/vi/KknNmrixt9Y/sddefault.jpg,MY FIRST TIME in Vietnam 🇻🇳 Ho Chi Minh City (...
1,https://i.ytimg.com/vi/h0Ia7Ddt88Q/sddefault.jpg,AMAZING FIRST IMPRESSIONS of Ho Chi Minh City ...
2,https://i.ytimg.com/vi/3xI5Ig2mXhc/sddefault.jpg,$45 LUXURY Airbnb Condo in Thao Dien 🇻🇳 DIGITA...
3,https://i.ytimg.com/vi/7EfERkvUEcY/sddefault.jpg,This is How They Treat you in VIETNAM 🇻🇳 STREE...
4,https://i.ytimg.com/vi/zfZsSSl0IMM/sddefault.jpg,Inside Rich Vietnam 🇻🇳 SHOCKING Thao Dien (Ho ...
...,...,...
891,https://i.ytimg.com/vi/sOm8DfbyF1s/sddefault.jpg,What to EXPECT hiking Table Mountain & Lion's ...
892,https://i.ytimg.com/vi/walaHUkoCGE/sddefault.jpg,Cape Town's Most Affluent Suburbs (where touri...
893,https://i.ytimg.com/vi/zSYCjfG0J3g/sddefault.jpg,The problem with Madeira
894,https://i.ytimg.com/vi/zp8FQFVyFOw/sddefault.jpg,EDINBURGH SCOTLAND TRAVEL GUIDE 2023


In [ ]:
# 만들어진 dataframe을 csv파일로 저장
vdf.to_csv("./URL_Title_List.csv")

## csv 파일에 있는 썸네일을 저장하고, 제목을 txt파일에 추가

In [18]:
from requests import get

In [9]:
loaded_df=pd.read_csv("./URL_Title_List.csv")

In [11]:
loaded_df.shape

(896, 3)

In [14]:
loaded_df.loc[0]['Thumbnail URL']

'https://i.ytimg.com/vi/KknNmrixt9Y/sddefault.jpg'

In [15]:
loaded_df.loc[0]['Title']

'MY FIRST TIME in Vietnam 🇻🇳 Ho Chi Minh City (Saigon) is REMARKABLE!'

In [23]:
def download(url, file_name):
    with open(file_name, "wb") as file:   # open in binary mode
        response = get(url)               # get request
        file.write(response.content)      # write to file
        
def append_to_txt_file(title_str, image_name, txt_file_path):
    f=open(txt_file_path, "a")
    f.write(image_name+" "+title_str+"\n")
    
txt_path="/home/host_data/ThumbCrawlImage/labels.txt"
for i in range(loaded_df.shape[0]):
    print(f"{i}----")
    print(loaded_df.loc[i]['Thumbnail URL'])
    print(loaded_df.loc[i]['Title'])
    download(loaded_df.loc[i]['Thumbnail URL'],"/home/host_data/ThumbCrawlImage/"+str(i)+".jpg")
#     append_to_txt_file(loaded_df.loc[i]['Title'], f"{i}.jpg",txt_path)
    print("-----")

0----
https://i.ytimg.com/vi/KknNmrixt9Y/sddefault.jpg
MY FIRST TIME in Vietnam 🇻🇳 Ho Chi Minh City (Saigon) is REMARKABLE!
-----
1----
https://i.ytimg.com/vi/h0Ia7Ddt88Q/sddefault.jpg
AMAZING FIRST IMPRESSIONS of Ho Chi Minh City (Saigon) Vietnam 🇻🇳
-----
2----
https://i.ytimg.com/vi/3xI5Ig2mXhc/sddefault.jpg
$45 LUXURY Airbnb Condo in Thao Dien 🇻🇳 DIGITAL NOMAD in Ho Chi Minh City (Saigon) Vietnam
-----
3----
https://i.ytimg.com/vi/7EfERkvUEcY/sddefault.jpg
This is How They Treat you in VIETNAM 🇻🇳 STREET FOOD in District 4 | Ho Chi Minh City (Saigon)
-----
4----
https://i.ytimg.com/vi/zfZsSSl0IMM/sddefault.jpg
Inside Rich Vietnam 🇻🇳 SHOCKING Thao Dien (Ho Chi Minh City) Area
-----
5----
https://i.ytimg.com/vi/AkSWY4rBc4A/sddefault.jpg
Vietnam's Biggest Banh Mi *FAMOUS* 🇻🇳 Ho Chi Minh City (Saigon)
-----
6----
https://i.ytimg.com/vi/FXg8ksz2KF4/sddefault.jpg
THE BEST AREA in Saigon...(by far?) | INCREDIBLE Ho Chi Minh City, Vietnam 🇻🇳
-----
7----
https://i.ytimg.com/vi/nYcV43jqKzI/sdd

KeyboardInterrupt: 